In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [2]:
##########################################################################
# Important Note: This notebook only uses one csv file to test the model #
##########################################################################

In [5]:
# assign processed data to data_files
cwd = os.getcwd()
data_dir = os.path.join(cwd, "data/")
data_files = [f for f in os.listdir(str(data_dir)) if f.endswith('csv')]

data = pd.read_csv(os.path.join(data_dir,data_files[0]))
data

,id,tdrift,tdrift50,tdrift10,rea,dcr,peakindex,peakvalue,tailslope,currentamp,lfpr,lq80,areagrowthrate,inflection point,risingedgeslope,energylabel,highavse,lowavse,truedcr,lq
0,650000,75.924,38.0,7.6,0.306251,5.647056e+05,1040,1236.0,-0.107739,0.006022,0.016692,137120.0,-525788.0,332,20.697813,449.096599,True,True,True,True
1,650001,82.917,41.5,8.3,0.610983,5.467540e+05,1049,1341.0,-0.110221,0.003851,0.018183,214469.0,-536552.0,337,17.853155,472.905768,True,True,True,True
2,650002,115.884,58.0,11.6,1.585428,2.801101e+05,1052,775.0,-0.056455,0.005939,0.023463,188754.0,-268393.5,337,4.624622,240.859403,True,True,True,True
3,650003,128.871,64.5,12.9,-0.471883,8.859788e+04,1068,242.0,-0.012898,0.004452,0.010622,109347.0,-64318.0,360,1.477667,56.433683,True,True,True,True
4,650004,95.904,48.0,9.6,0.903343,1.759048e+06,1060,3802.0,-0.358300,0.006815,0.019583,203822.0,-1714558.5,288,46.383844,1512.367517,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689996,649995,103.896,52.0,10.4,1.386256,2.888775e+05,1037,657.0,-0.055199,0.006327,0.021953,78290.0,-265790.5,308,5.563203,238.264153,True,True,True,True
1689997,649996,44.955,22.5,4.5,-0.555437,8.694774e+05,1023,1930.0,-0.169472,0.007386,0.017672,185348.0,-848319.5,271,50.753887,727.778128,False,False,True,False
1689998,649997,165.834,83.0,16.6,1.516233,2.478339e+05,1066,676.0,-0.050436,0.004362,0.024428,171916.0,-217168.0,300,2.899391,206.356597,True,True,True,True
1689999,649998,95.904,48.0,9.6,0.647120,3.457733e+06,1073,6729.0,-0.701681,0.005781,0.018728,348388.0,-3073543.0,342,85.845334,2615.179984,True,True,True,True


In [50]:
### EDA (To be continued)

# Drop columns needed for classification group
boolean_col = ['highavse','lowavse','truedcr','lq']
data_filtered = data.drop(columns=boolean_col+['id'])


# Find and Drop rows with missing values
print(data_filtered.isna().sum(axis=0))
data_filtered = data_filtered.dropna()

X = data_filtered.drop(columns=['energylabel'])
y = data_filtered['energylabel']

data_filtered



# Drop irrelevant features and feature with multicollinearity (To be Continued)
# See the last code block in this notbook for reference.

tdrift               0
tdrift50             0
tdrift10             0
rea                 36
dcr                  0
peakindex            0
peakvalue            0
tailslope            0
currentamp           0
lfpr                 0
lq80                 0
areagrowthrate       0
inflection point     0
risingedgeslope     36
energylabel          0
dtype: int64


,tdrift,tdrift50,tdrift10,rea,dcr,peakindex,peakvalue,tailslope,currentamp,lfpr,lq80,areagrowthrate,inflection point,risingedgeslope,energylabel
0,75.924,38.0,7.6,0.306251,5.647056e+05,1040,1236.0,-0.107739,0.006022,0.016692,137120.0,-525788.0,332,20.697813,449.096599
1,82.917,41.5,8.3,0.610983,5.467540e+05,1049,1341.0,-0.110221,0.003851,0.018183,214469.0,-536552.0,337,17.853155,472.905768
2,115.884,58.0,11.6,1.585428,2.801101e+05,1052,775.0,-0.056455,0.005939,0.023463,188754.0,-268393.5,337,4.624622,240.859403
3,128.871,64.5,12.9,-0.471883,8.859788e+04,1068,242.0,-0.012898,0.004452,0.010622,109347.0,-64318.0,360,1.477667,56.433683
4,95.904,48.0,9.6,0.903343,1.759048e+06,1060,3802.0,-0.358300,0.006815,0.019583,203822.0,-1714558.5,288,46.383844,1512.367517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689996,103.896,52.0,10.4,1.386256,2.888775e+05,1037,657.0,-0.055199,0.006327,0.021953,78290.0,-265790.5,308,5.563203,238.264153
1689997,44.955,22.5,4.5,-0.555437,8.694774e+05,1023,1930.0,-0.169472,0.007386,0.017672,185348.0,-848319.5,271,50.753887,727.778128
1689998,165.834,83.0,16.6,1.516233,2.478339e+05,1066,676.0,-0.050436,0.004362,0.024428,171916.0,-217168.0,300,2.899391,206.356597
1689999,95.904,48.0,9.6,0.647120,3.457733e+06,1073,6729.0,-0.701681,0.005781,0.018728,348388.0,-3073543.0,342,85.845334,2615.179984


In [56]:
# Train test split + standardization
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=523)
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.transform(X_test)

# GridCV to find best lambda
alpha_range = np.logspace(-5, 2.5, 100)
param_grid = {"alpha": alpha_range}
ridge = Ridge()
grid_search = GridSearchCV(estimator=ridge, param_grid={"alpha": alpha_range}, cv=5, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train_standardized, y_train)

best_alpha = grid_search.best_params_['alpha']
print(f"lambda: {best_alpha}")

# Train the Ridge Regression model with the best alpha
ridge_reg = Ridge(alpha=best_alpha)
ridge_reg.fit(X_train_standardized,y_train)
y_pred = ridge_reg.predict(X_test_standardized)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2= r2_score(y_test, y_pred)

lambda: 0.07305271542664456


In [59]:
mse,r2

(4943.309113510384, 0.9890277356961582)

In [60]:
np.mean(abs(y_pred - y_test))

16.38561521424413

In [38]:
data_filtered.corr()

,tdrift,tdrift50,tdrift10,rea,dcr,peakindex,peakvalue,tailslope,currentamp,lfpr,lq80,areagrowthrate,inflection point,risingedgeslope,energylabel
tdrift,1.000000,1.000000,1.000000,-0.353646,-0.119703,0.966766,-0.103934,0.111958,-0.130585,-0.286103,0.200552,0.118829,0.132879,-0.207463,-0.104135
tdrift50,1.000000,1.000000,1.000000,-0.353646,-0.119703,0.966766,-0.103934,0.111958,-0.130585,-0.286103,0.200552,0.118829,0.132879,-0.207463,-0.104135
tdrift10,1.000000,1.000000,1.000000,-0.353646,-0.119703,0.966766,-0.103934,0.111958,-0.130585,-0.286103,0.200552,0.118829,0.132879,-0.207463,-0.104135
rea,-0.353646,-0.353646,-0.353646,1.000000,0.028737,-0.431403,0.029845,-0.025223,0.245087,0.649042,-0.261714,-0.038562,-0.289455,-0.069715,0.028187
dcr,-0.119703,-0.119703,-0.119703,0.028737,1.000000,0.027153,0.997981,-0.996803,0.213606,0.089466,0.684838,-0.998767,-0.039834,0.917027,0.992463
peakindex,0.966766,0.966766,0.966766,-0.431403,0.027153,1.000000,0.042756,-0.033699,-0.107802,-0.276357,0.317137,-0.028126,0.148761,-0.058978,0.042922
peakvalue,-0.103934,-0.103934,-0.103934,0.029845,0.997981,0.042756,1.000000,-0.996964,0.198382,0.092079,0.711456,-0.999103,-0.045225,0.905183,0.994342
tailslope,0.111958,0.111958,0.111958,-0.025223,-0.996803,-0.033699,-0.996964,1.000000,-0.201219,-0.087887,-0.704436,0.996307,0.038172,-0.902061,-0.991746
currentamp,-0.130585,-0.130585,-0.130585,0.245087,0.213606,-0.107802,0.198382,-0.201219,1.000000,0.092553,-0.110800,-0.214132,-0.026983,0.244163,0.202332
lfpr,-0.286103,-0.286103,-0.286103,0.649042,0.089466,-0.276357,0.092079,-0.087887,0.092553,1.000000,-0.078496,-0.094299,-0.238815,0.024977,0.092440
